<a href="https://colab.research.google.com/github/LuuZwide/Supervised-Learning/blob/main/Dense_Neural_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import yfinance as yf
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/Functions/Preprocessing/Precossing_Functions_for_stock_chpters_1.ipynb" .
%run Precossing_Functions_for_stock_chpters_1.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve,roc_auc_score
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, MaxAbsScaler
import math
import datetime
from IPython.core.display import clear_output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import tensorflow.keras.backend as K
from keras.activations import elu

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=700)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())
ticker = yf.Ticker('IBM')
data = ticker.history(start=start_timestamp, end=end_timestamp, interval="1h")
len(data)

3370

In [ ]:
data.reset_index(drop = False, inplace = True)
data = data.drop(['Dividends', 'Stock Splits'], axis = 1)
prices = data['Close']
stock_prices = prices
len_of_data = len(prices)
window_size = 5
stock_dataset = Combine_dataframes(data ,prices, window_size,len_of_data, fast = 12, slow = 26)
#stock_dataset = stock_dataset.drop('next_close', axis = 1) # the next close price will be calculated with a neural net
chart = stock_dataset
chart = chart.reset_index(drop=True)
attribs = list(chart.columns)

macd:  Index(['MACD', 'Signal', 'Histogram'], dtype='object')
Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'moving_avg',
       'exm_avg', 'STOCHk_1_3_3', 'STOCHd_1_3_3', 'MACD', 'Signal',
       'Histogram'],
      dtype='object')


In [ ]:
# ticker = yf.Ticker("AAPL")
# apple_data = ticker.history(period="Max")
# apple_data = apple_data.drop(['Dividends','Stock Splits'] , axis = 1)
# apple_data.reset_index(drop = False, inplace = True)

# ticker = yf.Ticker("IBM")
# ibm_data = ticker.history(period="Max")
# ibm_data = ibm_data.drop(['Dividends','Stock Splits'] , axis = 1)
# ibm_data.reset_index(drop = False, inplace = True)

# ticker = yf.Ticker("GE")
# ge_data = ticker.history(period="Max")
# ge_data = ge_data.drop(['Dividends','Stock Splits'] , axis = 1)
# ge_data.reset_index(drop = False, inplace = True)

# ticker = yf.Ticker("amzn")
# amzn_data = ticker.history(period="Max")
# amzn_data = amzn_data.drop(['Dividends','Stock Splits'] , axis = 1)
# amzn_data.reset_index(drop = False, inplace = True)

# ticker = yf.Ticker("GOOGL")
# google_data = ticker.history(period="Max")
# google_data = google_data.drop(['Dividends','Stock Splits'] , axis = 1)
# google_data.reset_index(drop = False, inplace = True)


# stock_dict = {}
# stocks = ['IBM']

# for stock in stocks:
#   ticker = yf.Ticker(stock)
#   data = ticker.history(period="Max")
#   data = data.drop(['Dividends','Stock Splits'] , axis = 1)
#   data.reset_index(drop = False, inplace = True)
#   print(stock,'  ', len(data))

#   prices = data['Close']
#   len_of_data = len(prices)
#   window_size = 5
#   stock_dataset = Combine_dataframes(data ,prices, window_size,len_of_data, fast = 12, slow = 26)
#   #stock_dataset = stock_dataset.drop('next_close', axis = 1) # the next close price will be calculated with a neural net
#   stock_dict[stock] = stock_dataset
#   attribs = list(stock_dict[stock].columns)


# stock_pd = stock_dict['IBM']
# stock_pd = stock_pd.reset_index(drop = True)
# stock_pd.head()
# stock_pd.describe()


# stock_dataset = stock_pd


IBM    15447


In [ ]:
stock_dataset = chart
stock_dataset.head()

,Open,High,Low,Close,Volume,moving_avg,exm_avg,STOCHk_1_3_3,STOCHd_1_3_3,MACD,Signal,Histogram,next_close
0,138.699814,138.709366,138.539291,138.632889,0,0.000000,138.632889,0.000000,0.000000,0.000000,0.000000,0.000000,139.024857
1,138.632889,139.072662,138.565964,139.024857,217411,0.000000,138.868069,0.000000,0.000000,0.008166,0.004537,0.003629,138.705734
2,139.029633,139.077438,138.699814,138.705734,297436,0.000000,138.791174,49.055414,0.000000,0.001195,0.003167,-0.001972,138.881454
3,138.699814,138.929260,138.546844,138.881454,531611,0.000000,138.828675,59.877349,0.000000,0.003903,0.003416,0.000487,140.143402
4,138.871887,140.229446,138.718933,140.143402,558231,139.077667,139.333381,61.123481,56.685415,0.052023,0.017876,0.034148,139.655838


In [ ]:
y = stock_dataset['next_close']
X = stock_dataset.drop('next_close', axis = 1)
print(X)

            Open        High         Low       Close  Volume  moving_avg  \
0     138.699814  138.709366  138.539291  138.632889       0    0.000000   
1     138.632889  139.072662  138.565964  139.024857  217411    0.000000   
2     139.029633  139.077438  138.699814  138.705734  297436    0.000000   
3     138.699814  138.929260  138.546844  138.881454  531611    0.000000   
4     138.871887  140.229446  138.718933  140.143402  558231  139.077667   
...          ...         ...         ...         ...     ...         ...   
3365  128.770004  129.500000  128.720001  129.429993  869695  128.204001   
3366  129.460007  129.660004  129.104004  129.210007  604170  128.660002   
3367  129.240005  129.240005  128.660004  128.669998  347162  129.042001   
3368  128.690002  129.039993  128.619995  128.880005  525717  128.992001   
3369  128.860001  129.190002  128.740005  128.889999  906019  129.016000   

         exm_avg  STOCHk_1_3_3  STOCHd_1_3_3      MACD    Signal  Histogram  
0     138

In [ ]:
y_t = y
y_t.shape

(3370,)

In [ ]:
X.head(5)

,Open,High,Low,Close,Volume,moving_avg,exm_avg,STOCHk_1_3_3,STOCHd_1_3_3,MACD,Signal,Histogram
0,138.699814,138.709366,138.539291,138.632889,0,0.000000,138.632889,0.000000,0.000000,0.000000,0.000000,0.000000
1,138.632889,139.072662,138.565964,139.024857,217411,0.000000,138.868069,0.000000,0.000000,0.008166,0.004537,0.003629
2,139.029633,139.077438,138.699814,138.705734,297436,0.000000,138.791174,49.055414,0.000000,0.001195,0.003167,-0.001972
3,138.699814,138.929260,138.546844,138.881454,531611,0.000000,138.828675,59.877349,0.000000,0.003903,0.003416,0.000487
4,138.871887,140.229446,138.718933,140.143402,558231,139.077667,139.333381,61.123481,56.685415,0.052023,0.017876,0.034148


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)
attribs = list(x_train.columns)
#attribs = list(['Open', 'High', 'Low', 'Close', 'moving_avg', 'exm_avg'])
print(len(x_train))
print(len(x_test))

2696
674


In [ ]:
y

99.57503509521484

In [ ]:
print(x_train)

            Open       High        Low      Close   Volume  moving_avg  \
0       1.609601   1.609601   1.591516   1.591516   407940    0.000000   
1       1.591517   1.605429   1.591517   1.605429   305955    0.000000   
2       1.605428   1.605428   1.588733   1.589429   274575    0.000000   
3       1.587343   1.587343   1.555346   1.558128   384405    0.000000   
4       1.556738   1.556738   1.516393   1.528914   572685    1.574683   
...          ...        ...        ...        ...      ...         ...   
12352  98.581091  98.721344  97.818886  98.196938  5599343   96.759097   
12353  98.434768  98.892099  98.087208  98.215256  5102702   97.401802   
12354  98.203040  98.733538  96.751787  97.081070  7034978   97.854254   
12355  97.062775  98.782318  96.757890  98.782318  7528271   98.143286   
12356  98.849413  99.965291  98.782338  99.733574  6099540   98.401831   

         exm_avg  STOCHk_5_3_3  STOCHd_5_3_3  MACD_12_26_9  MACDh_12_26_9  \
0       1.591516      0.000000    

In [ ]:
data_pipeline = Pipeline(
    (
        ('ConvertDataFrame',DataFrameSelector(attribs)),
        ('Scaler', MinMaxScaler())
    )
)

x_train_prepared = data_pipeline.fit_transform(X)
print('x_train shape: ',x_train_prepared.shape)
print('y_train shape: ',y_train.shape)

x_train shape:  (3370, 12)
y_train shape:  (2696,)


In [ ]:
y

0       139.024857
1       138.705734
2       138.881454
3       140.143402
4       139.655838
           ...    
3365    129.210007
3366    128.669998
3367    128.880005
3368    128.889999
3369    128.889999
Name: next_close, Length: 3370, dtype: float64

In [ ]:
print(x_train_prepared)

[[0.6426196  0.61080363 0.64759014 ... 0.5062416  0.50737936 0.50237604]
 [0.64080493 0.62055449 0.64831046 ... 0.50746163 0.50814475 0.50385073]
 [0.65156262 0.62068268 0.65192525 ... 0.50642009 0.50791367 0.50157463]
 ...
 [0.38611723 0.35664655 0.3807879  ... 0.58242971 0.54458429 0.61997606]
 [0.37120392 0.35127823 0.37970742 ... 0.58714175 0.5554142  0.60670816]
 [0.37581342 0.35530447 0.38294845 ... 0.58992447 0.5647066  0.59189617]]


In [ ]:
def plot_learning_rate(history):
  lr = history.history['lr']
  epochs = range(1, len(lr) +1)
  plt.plot(epochs, lr)
  plt.yscale('log')
  plt.title('Learning Rate per Epoch')
  plt.xlabel('Epoch')
  plt.ylabel('Learning Rate')
  plt.show()

In [ ]:
def inverse_time_decay(epoch, lr):
  k = 1
  lr = lr / (1 + k * epoch)
  return lr

In [ ]:
def exponential_decay(epoch, lr):
  initial_lr = lr
  alpha = 0.0001
  min_lr = 0.001
  max_decay = 0.2
  decay = alpha * epoch
  if (decay > max_decay):
    decay = max_decay
  new_lr = initial_lr * np.exp(-alpha * epoch)
  if (new_lr < min_lr):
    new_lr = min_lr
  return new_lr

In [ ]:
lr = 0.001
dropout_rate = 0.15


model = keras.Sequential([
    keras.layers.Dense(units = 12, activation = 'relu', use_bias = True, kernel_initializer = 'glorot_uniform' ),
    keras.layers.Dense(units = 124, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 64, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 32, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 16, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 1)
])


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
    loss = tf.losses.MeanAbsoluteError(reduction = 'auto', name = 'mean_absolute_error'),
    metrics = ['mean_squared_error']
)

early_stopping = EarlyStopping(monitor = 'mean_squared_error', patience = 15)
#callbacks = [LearningRateScheduler(exponential_decay),early_stopping ]

history = model.fit(
                    x = x_train_prepared,
                    y = [y_t],
                    batch_size = 128,
                    epochs = 850,
                    #callbacks = [early_stopping]
                  )

Epoch 1/850
27/27 [==============================] - 1s 3ms/step - loss: 131.2998 - mean_squared_error: 17290.4727
Epoch 2/850
27/27 [==============================] - 0s 3ms/step - loss: 122.1126 - mean_squared_error: 14996.9961
Epoch 3/850
27/27 [==============================] - 0s 3ms/step - loss: 50.5108 - mean_squared_error: 3634.5303
Epoch 4/850
27/27 [==============================] - 0s 3ms/step - loss: 10.7654 - mean_squared_error: 177.4201
Epoch 5/850
27/27 [==============================] - 0s 3ms/step - loss: 7.5225 - mean_squared_error: 86.8635
Epoch 6/850
27/27 [==============================] - 0s 3ms/step - loss: 5.6483 - mean_squared_error: 50.4831
Epoch 7/850
27/27 [==============================] - 0s 3ms/step - loss: 3.7393 - mean_squared_error: 24.1415
Epoch 8/850
27/27 [==============================] - 0s 3ms/step - loss: 2.0675 - mean_squared_error: 10.0842
Epoch 9/850
27/27 [==============================] - 0s 3ms/step - loss: 1.3149 - mean_squared_error: 5.7

In [ ]:
mse = model.evaluate(x_train_prepared, [y_t], verbose = 0)
print('mse: ', mse)
rmse = np.sqrt(mse)
print('rmse: ', rmse)

mse:  [0.5402992367744446, 0.624788761138916]
rmse:  [0.7350505 0.7904358]


In [ ]:
model.save('/content/drive/MyDrive/Models/Prediction Model/Dense_pred.h5')

In [ ]:
print(x_train_prepared.shape)

(15447, 12)


In [ ]:

predictions = model.predict(x_train_prepared[-50:])



for x,y in zip(predictions, y_t[-50:]):
  print(x, '  ',y)

2/2 [==============================] - 0s 5ms/step
[125.45686]    125.71499633789062
[125.465866]    125.58000183105469
[125.34711]    125.5
[125.26193]    125.625
[125.37811]    126.0604019165039
[125.78975]    125.91500091552734
[125.61597]    126.16000366210938
[125.87986]    128.05499267578125
[127.789116]    127.6614990234375
[127.34178]    127.3949966430664
[127.07427]    127.69999694824219
[127.438644]    127.62999725341797
[127.42021]    127.61000061035156
[127.32001]    127.26000213623047
[126.90155]    127.69999694824219
[127.392975]    127.69000244140625
[127.35394]    127.58000183105469
[127.26004]    127.38249969482422
[127.07425]    127.66500091552734
[127.41339]    127.58999633789062
[127.329575]    127.5199966430664
[127.24058]    127.26000213623047
[126.97455]    127.86499786376953
[127.59999]    128.52000427246094
[128.24532]    128.70680236816406
[128.44559]    127.8949966430664
[127.649796]    128.52999877929688
[128.3277]    128.1699981689453
[127.94504]    126.419